# 作业
1. 请总结`MessagePassing`基类的运行流程。
MessagePassing的__init__()函数中会检查子类中message_and_aggregate方法是否被定义，检查的结果放在self.fuse中。


在子类的forward函数中会调用`MessagePassing`的propogate函数，propogate的运行逻辑如下：

edge_index是SparseTensor类型，且子类中message_and_aggregate方法被定义,且self.\_\_explain\_\_为False（？？）

则调用message_and_aggregate和update函数，在调用这两个函数时会使用self.inspector.distribute函数调取参数

如果edge_index为Tensor类型或者子类中message_and_aggregate方法未被定义

则依次调用message, aggregate, update函数，调用时也会使用self.inspector.distribute函数调取参数

2. 请复现一个一层的图神经网络的构造，总结通过继承`MessagePassing`基类来构造自己的图神经网络类的规范。

In [ ]:
from torch_geometric.datasets import Planetoid
import torch
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_sparse import SparseTensor


class GCNConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add', flow='source_to_target')
        # "Add" aggregation (Step 5).
        # flow='source_to_target' 表示消息从源节点传播到目标节点
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]

        # Step 1: Add self-loops to the adjacency matrix.
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Linearly transform node feature matrix.
        x = self.lin(x)

        # Step 3: Compute normalization.
        row, col = edge_index
        deg = degree(col, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4-5: Start propagating messages.
        adjmat = SparseTensor(row=edge_index[0], col=edge_index[1], value=torch.ones(edge_index.shape[1]))
        # 此处传的不再是edge_idex，而是SparseTensor类型的Adjancency Matrix
        return self.propagate(adjmat, x=x, norm=norm, deg=deg.view((-1, 1)))

    def message(self, x_j, norm, deg_i):
        # x_j has shape [E, out_channels]
        # deg_i has shape [E, 1]
        # Step 4: Normalize node features.
        return norm.view(-1, 1) * x_j * deg_i

    def aggregate(self, inputs, index, ptr, dim_size):
        return super().aggregate(inputs, index, ptr=ptr, dim_size=dim_size)

    def message_and_aggregate(self, adj_t, x, norm):
        msg_kwargs = self.inspector.distribute('message', coll_dict)
        out = self.message(**msg_kwargs)
        # 此处省略n行代码
        aggr_kwargs = self.inspector.distribute('aggregate', coll_dict)
        return self.aggregate(out, **aggr_kwargs)

    def update(self, inputs, deg):
        print(deg)
        return inputs


dataset = Planetoid(root='dataset', name='Cora')
data = dataset[0]

net = GCNConv(data.num_features, 64)
h_nodes = net(data.x, data.edge_index)
# print(h_nodes.shape)



# 消息传递图神经网络笔记
## 消息传递范式介绍
**节点信息**（是否是data.x?)经过变换得到**节点表征**
## Pytorch Geometric中的`MessagePassing`基类





